# 2/4/5호선 종합 분석
## 역간 소요시간 × 배차간격 = 승객 경험 지표

이 노트북은 서울 지하철 2호선, 4호선, 5호선의 역간 소요시간과 배차간격을 분석하여 서비스 개선이 필요한 구간을 식별합니다.

### 분석 내용
1. 역간 소요시간 계산 및 분석
2. 역별 배차간격 계산 및 분석
3. 복합지표(소요시간 × 배차간격) 계산
4. 서비스 개선 우선순위 도출
5. 호선별 비교 분석
6. 시각화 및 결과 저장

In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# 파일 경로
CSV_PATH = r'..\data\input\서울교통공사_서울 도시철도 열차운행시각표_20250704.csv'
OUTPUT_DIR = r'..\data\output\4_2_4_5호선_종합'

# 호선 색상
LINE_COLORS = {2: '#00D84A', 4: '#00A5DE', 5: '#996CAC'}

print('✅ 환경 설정 완료')

✅ 환경 설정 완료


## 1. 데이터 로딩 및 전처리

In [34]:
# CSV 파일 로드
df_raw = pd.read_csv(CSV_PATH, encoding='cp949')

# 2, 4, 5호선만 필터링
df_all = df_raw[df_raw['호선'].isin([2, 4, 5])].copy()

# 일반 열차만 (급행 제외)
df_all = df_all[df_all['급행여부'] == 0].copy()

# datetime 변환
df_all['열차도착시간_dt'] = pd.to_datetime(df_all['열차도착시간'], format='%H:%M:%S', errors='coerce')
df_all['열차출발시간_dt'] = pd.to_datetime(df_all['열차출발시간'], format='%H:%M:%S', errors='coerce')

# 시간대 추출
df_all['시간대'] = df_all['열차도착시간_dt'].dt.hour

# 검증
assert df_all['호선'].nunique() == 3, '호선이 3개가 아닙니다'
assert all(df_all['호선'].isin([2, 4, 5])), '2, 4, 5호선 이외의 데이터가 포함되어 있습니다'

print(f'✅ 데이터 로딩 완료')
print(f'   총 데이터: {len(df_all):,}행')
print(f'   고유 열차 수: {df_all["열차코드"].nunique():,}대')
print(f'   고유 역 수: {df_all["역사명"].nunique():,}개')
print(f'   호선별 데이터 수:')
for line in [2, 4, 5]:
    count = len(df_all[df_all['호선'] == line])
    print(f'      {line}호선: {count:,}행')

df_all.head(10)

✅ 데이터 로딩 완료
   총 데이터: 160,008행
   고유 열차 수: 2,372대
   고유 역 수: 150개
   호선별 데이터 수:
      2호선: 61,861행
      4호선: 48,163행
      5호선: 49,984행


,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역,열차도착시간_dt,열차출발시간_dt,시간대
88431,88432,2,200,까치산,DAY,DOWN,0,5501,NaN,05:30:00,까치산,신도림,NaT,1900-01-01 05:30:00,NaN
88432,88433,2,200,까치산,DAY,DOWN,0,5503,NaN,05:46:00,까치산,신도림,NaT,1900-01-01 05:46:00,NaN
88433,88434,2,200,까치산,DAY,DOWN,0,5505,NaN,06:00:00,까치산,신도림,NaT,1900-01-01 06:00:00,NaN
88434,88435,2,200,까치산,DAY,DOWN,0,5507,NaN,06:14:00,까치산,신도림,NaT,1900-01-01 06:14:00,NaN
88435,88436,2,200,까치산,DAY,DOWN,0,5509,NaN,06:31:00,까치산,신도림,NaT,1900-01-01 06:31:00,NaN
88436,88437,2,200,까치산,DAY,DOWN,0,5511,NaN,06:48:00,까치산,신도림,NaT,1900-01-01 06:48:00,NaN
88437,88438,2,200,까치산,DAY,DOWN,0,5513,NaN,06:59:00,까치산,신도림,NaT,1900-01-01 06:59:00,NaN
88438,88439,2,200,까치산,DAY,DOWN,0,5515,NaN,07:09:00,까치산,신도림,NaT,1900-01-01 07:09:00,NaN
88439,88440,2,200,까치산,DAY,DOWN,0,5517,NaN,07:18:00,까치산,신도림,NaT,1900-01-01 07:18:00,NaN
88440,88441,2,200,까치산,DAY,DOWN,0,5519,NaN,07:28:00,까치산,신도림,NaT,1900-01-01 07:28:00,NaN


## 2. 역간 소요시간 계산

각 열차의 연속된 정차역 사이의 이동 시간을 계산합니다.

In [35]:
def calculate_inter_station_time(df):
    """
    역간 소요시간 계산 (검증된 패턴 사용)
    """
    travel_times = []
    
    for (train_code, day_type), group in df.groupby(['열차코드', '주중주말']):
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        for i in range(len(group) - 1):
            current = group.iloc[i]
            next_station = group.iloc[i + 1]
            
            if pd.notna(current['열차출발시간_dt']) and pd.notna(next_station['열차도착시간_dt']):
                travel_time = (next_station['열차도착시간_dt'] - current['열차출발시간_dt']).total_seconds() / 60
                
                if 0 < travel_time < 60:
                    travel_times.append({
                        '호선': current['호선'],
                        '출발역명': current['역사명'],
                        '도착역명': next_station['역사명'],
                        '소요시간_분': travel_time,
                        '열차코드': train_code,
                        '주중주말': day_type,
                        '방향': current['방향']
                    })
    
    return pd.DataFrame(travel_times)

print('⏱️ 역간 소요시간 계산 중...')
df_travel = calculate_inter_station_time(df_all)

print(f'✅ {len(df_travel):,}개의 역간 이동 기록 생성')
df_travel.head(10)

⏱️ 역간 소요시간 계산 중...
✅ 147,400개의 역간 이동 기록 생성


,호선,출발역명,도착역명,소요시간_분,열차코드,주중주말,방향
0,5,아차산,광나루,1.833333,5001,DAY,DOWN
1,5,광나루,천호,2.166667,5001,DAY,DOWN
2,5,천호,강동,1.166667,5001,DAY,DOWN
3,5,강동,길동,1.500000,5001,DAY,DOWN
4,5,길동,굽은다리,1.166667,5001,DAY,DOWN
5,5,굽은다리,명일,1.166667,5001,DAY,DOWN
6,5,명일,고덕,1.666667,5001,DAY,DOWN
7,5,고덕,상일동,1.666667,5001,DAY,DOWN
8,5,상일동,강일,1.833333,5001,DAY,DOWN
9,5,강일,미사,1.833333,5001,DAY,DOWN


In [36]:
# ⚠️ 중요: 통계 집계 후 즉시 컬럼 플래팅
travel_stats = df_travel.groupby(['호선', '출발역명', '도착역명', '방향', '주중주말']).agg({
    '소요시간_분': ['mean', 'std', 'min', 'max', 'count']
}).round(2)

# ✅ 컬럼 즉시 플래팅 (MultiIndex 제거)
travel_stats.columns = ['평균_분', '표준편차', '최소_분', '최대_분', '측정횟수']
travel_stats = travel_stats.reset_index()
travel_stats['구간'] = travel_stats['출발역명'] + ' → ' + travel_stats['도착역명']

# 검증
assert '평균_분' in travel_stats.columns
assert '표준편차' in travel_stats.columns
print(f'✅ 역간 소요시간 통계 계산 완료: {len(travel_stats):,}개 구간')
print(f'   컬럼: {list(travel_stats.columns)}')

# 주중 데이터만 추출 및 컬럼명 변경
travel_weekday = travel_stats[travel_stats['주중주말'] == 'DAY'].copy()
travel_weekday = travel_weekday.rename(columns={'평균_분': '평균소요시간_분'})

print(f'\n주중 데이터: {len(travel_weekday):,}개 구간')
print(f'평균 소요시간 통계:')
print(f'  전체 평균: {travel_weekday["평균소요시간_분"].mean():.2f}분')
print(f'  최소: {travel_weekday["평균소요시간_분"].min():.2f}분')
print(f'  최대: {travel_weekday["평균소요시간_분"].max():.2f}분')

travel_weekday.head(20)

✅ 역간 소요시간 통계 계산 완료: 1,071개 구간
   컬럼: ['호선', '출발역명', '도착역명', '방향', '주중주말', '평균_분', '표준편차', '최소_분', '최대_분', '측정횟수', '구간']

주중 데이터: 339개 구간
평균 소요시간 통계:
  전체 평균: 1.85분
  최소: 0.17분
  최대: 58.17분


,호선,출발역명,도착역명,방향,주중주말,평균소요시간_분,표준편차,최소_분,최대_분,측정횟수,구간
0,2,강남,교대,OUT,DAY,1.50,0.0,1.50,1.50,234,강남 → 교대
3,2,강남,역삼,IN,DAY,1.00,0.0,1.00,1.00,235,강남 → 역삼
6,2,강변,구의,IN,DAY,1.00,0.0,1.00,1.00,234,강변 → 구의
9,2,강변,잠실나루,OUT,DAY,2.00,0.0,2.00,2.00,235,강변 → 잠실나루
12,2,건대입구,구의,OUT,DAY,2.00,0.0,2.00,2.00,235,건대입구 → 구의
15,2,건대입구,성수,IN,DAY,1.50,0.0,1.50,1.50,234,건대입구 → 성수
18,2,교대,강남,IN,DAY,1.50,0.0,1.50,1.50,236,교대 → 강남
21,2,교대,서초,OUT,DAY,1.00,0.0,1.00,1.00,234,교대 → 서초
24,2,구로디지털단지,대림,OUT,DAY,1.50,0.0,1.50,1.50,231,구로디지털단지 → 대림
27,2,구로디지털단지,신대방,IN,DAY,1.50,0.0,1.50,1.50,236,구로디지털단지 → 신대방


## 3. 배차간격 계산

각 역에서 연속된 열차가 도착하는 시간 간격을 계산합니다.

In [37]:
def calculate_headway_by_station(df):
    """
    역별 배차간격 계산 (검증된 패턴 사용)
    """
    headways = []
    
    for (line, station, direction, day_type), group in df.groupby(['호선', '역사명', '방향', '주중주말']):
        group = group[group['열차도착시간_dt'].notna()].copy()
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        if len(group) > 1:
            group['배차간격_분'] = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
            valid_headways = group['배차간격_분'].dropna()
            valid_headways = valid_headways[(valid_headways > 1) & (valid_headways < 60)]
            
            if len(valid_headways) > 0:
                headways.append({
                    '호선': line,
                    '역사명': station,
                    '방향': direction,
                    '주중주말': day_type,
                    '평균배차간격_분': valid_headways.mean(),
                    '최소배차간격_분': valid_headways.min(),
                    '최대배차간격_분': valid_headways.max(),
                    '표준편차': valid_headways.std(),
                    '측정횟수': len(valid_headways)
                })
    
    return pd.DataFrame(headways)

print('⏱️ 배차간격 계산 중...')
df_headway = calculate_headway_by_station(df_all)

# 검증
assert '평균배차간격_분' in df_headway.columns
print(f'✅ 배차간격 계산 완료: {len(df_headway):,}개 역-방향-주중주말 조합')
print(f'   컬럼: {list(df_headway.columns)}')

# 주중 데이터
headway_weekday = df_headway[df_headway['주중주말'] == 'DAY'].copy()

print(f'\n주중 데이터: {len(headway_weekday):,}개')
print(f'평균 배차간격 통계:')
print(f'  전체 평균: {headway_weekday["평균배차간격_분"].mean():.2f}분')
print(f'  최소: {headway_weekday["평균배차간격_분"].min():.2f}분')
print(f'  최대: {headway_weekday["평균배차간격_분"].max():.2f}분')

df_headway.head(20)

⏱️ 배차간격 계산 중...
✅ 배차간격 계산 완료: 933개 역-방향-주중주말 조합
   컬럼: ['호선', '역사명', '방향', '주중주말', '평균배차간격_분', '최소배차간격_분', '최대배차간격_분', '표준편차', '측정횟수']

주중 데이터: 311개
평균 배차간격 통계:
  전체 평균: 6.71분
  최소: 4.64분
  최대: 14.63분


,호선,역사명,방향,주중주말,평균배차간격_분,최소배차간격_분,최대배차간격_분,표준편차,측정횟수
0,2,강남,IN,DAY,4.657447,2.0,13.0,1.752501,235
1,2,강남,IN,END,5.652850,3.0,16.5,1.334129,193
2,2,강남,IN,SAT,5.652850,3.0,16.5,1.334129,193
3,2,강남,OUT,DAY,4.710300,2.0,16.5,1.690102,233
4,2,강남,OUT,END,5.558673,2.5,16.5,1.369213,196
5,2,강남,OUT,SAT,5.558673,2.5,16.5,1.369213,196
6,2,강변,IN,DAY,4.675966,2.0,20.0,1.975917,233
7,2,강변,IN,END,5.668394,2.5,19.5,1.507407,193
8,2,강변,IN,SAT,5.668394,2.5,19.5,1.507407,193
9,2,강변,OUT,DAY,4.688034,2.0,10.0,1.526204,234


## 4. 복합지표 계산: 소요시간 × 배차간격

각 구간의 소요시간과 출발역의 배차간격을 곱하여 승객 경험 지표를 생성합니다.

In [38]:
# Merge 패턴 (검증됨)
df_combined = travel_weekday.merge(
    headway_weekday[['호선', '역사명', '방향', '평균배차간격_분']],
    left_on=['호선', '출발역명', '방향'],
    right_on=['호선', '역사명', '방향'],
    how='left'
)

# ⚠️ 중요: 중복 컬럼 즉시 제거
df_combined = df_combined.drop('역사명', axis=1)

# 복합지표 계산
df_combined['복합지표'] = df_combined['평균소요시간_분'] * df_combined['평균배차간격_분']
df_combined = df_combined.dropna(subset=['평균배차간격_분', '복합지표'])

# 검증
assert '평균소요시간_분' in df_combined.columns
assert '평균배차간격_분' in df_combined.columns
assert '복합지표' in df_combined.columns
assert '역사명' not in df_combined.columns, '중복 컬럼이 제거되지 않았습니다'

print(f'✅ 복합지표 계산 완료: {len(df_combined):,}개 구간')
print(f'   컬럼: {list(df_combined.columns)}')
print(f'\n복합지표 통계:')
print(f'  평균: {df_combined["복합지표"].mean():.2f}')
print(f'  중앙값: {df_combined["복합지표"].median():.2f}')
print(f'  최소: {df_combined["복합지표"].min():.2f}')
print(f'  최대: {df_combined["복합지표"].max():.2f}')

# Top 10 출력
print(f'\n🚨 복합지표 Top 10 (주중):')
df_combined.nlargest(10, '복합지표')[['호선', '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표']]

✅ 복합지표 계산 완료: 339개 구간
   컬럼: ['호선', '출발역명', '도착역명', '방향', '주중주말', '평균소요시간_분', '표준편차', '최소_분', '최대_분', '측정횟수', '구간', '평균배차간격_분', '복합지표']

복합지표 통계:
  평균: 12.96
  중앙값: 9.30
  최소: 1.79
  최대: 326.97

🚨 복합지표 Top 10 (주중):


,호선,구간,방향,평균소요시간_분,평균배차간격_분,복합지표
229,5,공덕 → 신정네거리,DOWN,58.17,5.620898,326.967642
146,4,별내별가람 → 오남,UP,6.50,14.606667,94.943333
182,4,오남 → 별내별가람,DOWN,6.50,14.605263,94.934211
145,4,별내별가람 → 불암산,DOWN,5.00,14.500000,72.500000
152,4,산본 → 상록수,DOWN,8.00,8.732000,69.856000
274,5,상일동 → 하남풍산,DOWN,6.17,10.906463,67.292874
183,4,오남 → 진접,UP,2.50,14.606667,36.516667
148,4,불암산 → 상계,DOWN,2.50,14.500000,36.250000
151,4,산본 → 금정,UP,4.01,8.818548,35.362379
180,4,안산 → 정왕,DOWN,4.00,8.678862,34.715447


## 5. 개선 우선순위 계산

복합지표를 기반으로 서비스 개선이 필요한 구간의 우선순위를 계산합니다.

In [39]:
df_priority = df_combined.copy()

# 0-100 스케일 정규화
df_priority['소요시간_점수'] = (
    (df_priority['평균소요시간_분'] - df_priority['평균소요시간_분'].min()) /
    (df_priority['평균소요시간_분'].max() - df_priority['평균소요시간_분'].min()) * 100
).round(2)

df_priority['배차간격_점수'] = (
    (df_priority['평균배차간격_분'] - df_priority['평균배차간격_분'].min()) /
    (df_priority['평균배차간격_분'].max() - df_priority['평균배차간격_분'].min()) * 100
).round(2)

# 종합 점수 (가중 평균)
df_priority['종합점수'] = (
    df_priority['소요시간_점수'] * 0.3 +
    df_priority['배차간격_점수'] * 0.3 +
    ((df_priority['복합지표'] - df_priority['복합지표'].min()) /
     (df_priority['복합지표'].max() - df_priority['복합지표'].min()) * 100) * 0.4
).round(2)

# 개선 방안 추천
def recommend_improvement(row):
    if row['배차간격_점수'] > row['소요시간_점수']:
        return '배차 증편 우선'
    elif row['소요시간_점수'] > row['배차간격_점수']:
        return '선로/신호 개선 우선'
    else:
        return '종합 개선 필요'

df_priority['개선방안'] = df_priority.apply(recommend_improvement, axis=1)

# Top 30
priority_top30 = df_priority.nlargest(30, '종합점수')[[
    '호선', '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표',
    '소요시간_점수', '배차간격_점수', '종합점수', '개선방안'
]].reset_index(drop=True)

print('🎯 개선 우선순위 Top 30 (주중)')
print('=' * 100)
print('종합점수 = 소요시간(30%) + 배차간격(30%) + 복합지표(40%)')
print('=' * 100)
priority_top30

🎯 개선 우선순위 Top 30 (주중)
종합점수 = 소요시간(30%) + 배차간격(30%) + 복합지표(40%)


,호선,구간,방향,평균소요시간_분,평균배차간격_분,복합지표,소요시간_점수,배차간격_점수,종합점수,개선방안
0,5,공덕 → 신정네거리,DOWN,58.17,5.620898,326.967642,100.00,9.82,72.95,선로/신호 개선 우선
1,4,별내별가람 → 오남,UP,6.50,14.606667,94.943333,10.91,100.00,44.73,배차 증편 우선
2,4,오남 → 별내별가람,DOWN,6.50,14.605263,94.934211,10.91,99.99,44.73,배차 증편 우선
3,4,별내별가람 → 불암산,DOWN,5.00,14.500000,72.500000,8.33,98.93,40.88,배차 증편 우선
4,4,오남 → 진접,UP,2.50,14.606667,36.516667,4.02,100.00,35.48,배차 증편 우선
5,4,불암산 → 상계,DOWN,2.50,14.500000,36.250000,4.02,98.93,35.12,배차 증편 우선
6,5,상일동 → 하남풍산,DOWN,6.17,10.906463,67.292874,10.34,62.86,30.02,배차 증편 우선
7,5,하남풍산 → 미사,UP,2.50,12.247191,30.617978,4.02,76.32,27.65,배차 증편 우선
8,5,강일 → 상일동,UP,1.83,12.247191,22.412360,2.86,76.32,26.29,배차 증편 우선
9,5,미사 → 강일,UP,1.83,12.247191,22.412360,2.86,76.32,26.29,배차 증편 우선


## 6. 호선별 요약 통계

In [40]:
line_summary = []

for line in [2, 4, 5]:
    line_travel = travel_weekday[travel_weekday['호선'] == line]
    line_headway = headway_weekday[headway_weekday['호선'] == line]
    line_combined = df_combined[df_combined['호선'] == line]
    
    summary = {
        '호선': line,
        '평균 소요시간(분)': line_travel['평균소요시간_분'].mean().round(2) if len(line_travel) > 0 else 0,
        '평균 배차간격(분)': line_headway['평균배차간격_분'].mean().round(2) if len(line_headway) > 0 else 0,
        '평균 복합지표': line_combined['복합지표'].mean().round(2) if len(line_combined) > 0 else 0,
        '고유 구간 수': len(line_travel),
        '고유 역 수': line_headway['역사명'].nunique() if len(line_headway) > 0 else 0
    }
    line_summary.append(summary)

df_line_summary = pd.DataFrame(line_summary)

print('📊 호선별 요약 통계 (주중):')
df_line_summary

📊 호선별 요약 통계 (주중):


,호선,평균 소요시간(분),평균 배차간격(분),평균 복합지표,고유 구간 수,고유 역 수
0,2,1.46,5.59,8.67,111,51
1,4,2.20,7.26,17.36,102,51
2,5,1.91,7.24,13.18,126,56


## 7. 시각화

### 7.1 호선별 소요시간 Top 10

In [41]:
# 호선별 Top 10
fig = go.Figure()

for line in [2, 4, 5]:
    line_data = travel_weekday[travel_weekday['호선'] == line].nlargest(10, '평균소요시간_분')
    fig.add_trace(go.Bar(
        name=f'{line}호선',
        y=line_data['구간'],
        x=line_data['평균소요시간_분'],
        orientation='h',
        marker_color=LINE_COLORS[line]
    ))

fig.update_layout(
    title='호선별 평균 소요시간 Top 10 (주중)',
    xaxis_title='평균 소요시간 (분)',
    yaxis_title='구간',
    height=800,
    barmode='group'
)

fig.show()

### 7.2 호선별 배차간격 Top 10

In [42]:
fig = go.Figure()

for line in [2, 4, 5]:
    line_data = headway_weekday[headway_weekday['호선'] == line].nlargest(10, '평균배차간격_분')
    line_data['역-방향'] = line_data['역사명'] + ' (' + line_data['방향'] + ')'
    fig.add_trace(go.Bar(
        name=f'{line}호선',
        y=line_data['역-방향'],
        x=line_data['평균배차간격_분'],
        orientation='h',
        marker_color=LINE_COLORS[line]
    ))

fig.update_layout(
    title='호선별 평균 배차간격 Top 10 (주중)',
    xaxis_title='평균 배차간격 (분)',
    yaxis_title='역 (방향)',
    height=800,
    barmode='group'
)

fig.show()

### 7.3 복합지표 Top 30

In [43]:
top30_viz = df_combined.nlargest(30, '복합지표').copy()
colors = [LINE_COLORS[line] for line in top30_viz['호선']]

fig = px.bar(
    top30_viz,
    x='복합지표',
    y='구간',
    orientation='h',
    title='복합지표 Top 30 (주중)<br><sub>소요시간 × 배차간격</sub>',
    labels={'복합지표': '복합지표 (분²)', '구간': '역 구간'},
    color='호선',
    color_discrete_map=LINE_COLORS,
    hover_data={'평균소요시간_분': ':.2f', '평균배차간격_분': ':.2f'}
)

fig.update_layout(
    height=900,
    yaxis={'categoryorder': 'total ascending'}
)

fig.show()

### 7.4 소요시간 vs 배차간격 산점도

In [44]:
fig = px.scatter(
    df_combined,
    x='평균소요시간_분',
    y='평균배차간격_분',
    size='복합지표',
    color='호선',
    color_discrete_map=LINE_COLORS,
    hover_data=['구간', '방향'],
    title='소요시간 vs 배차간격 (주중)<br><sub>크기 = 복합지표</sub>',
    labels={
        '평균소요시간_분': '평균 소요시간 (분)',
        '평균배차간격_분': '평균 배차간격 (분)'
    }
)

# 중앙값 선
median_travel = df_combined['평균소요시간_분'].median()
median_headway = df_combined['평균배차간격_분'].median()

fig.add_hline(y=median_headway, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=median_travel, line_dash="dash", line_color="gray", opacity=0.5)

fig.show()

### 7.5 호선별 비교 차트

In [45]:
# 막대 차트
fig = go.Figure()

fig.add_trace(go.Bar(
    name='평균 소요시간',
    x=df_line_summary['호선'],
    y=df_line_summary['평균 소요시간(분)'],
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    name='평균 배차간격',
    x=df_line_summary['호선'],
    y=df_line_summary['평균 배차간격(분)'],
    marker_color='lightcoral'
))

fig.update_layout(
    title='호선별 평균 소요시간 및 배차간격 비교 (주중)',
    xaxis_title='호선',
    yaxis_title='시간 (분)',
    barmode='group',
    height=500
)

fig.show()

### 7.6 개선 우선순위 Top 10 (호선별)

In [46]:
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['2호선', '4호선', '5호선']
)

for idx, line in enumerate([2, 4, 5], 1):
    line_priority = df_priority[df_priority['호선'] == line].nlargest(10, '종합점수')
    
    fig.add_trace(
        go.Bar(
            x=line_priority['종합점수'],
            y=line_priority['구간'],
            orientation='h',
            marker_color=LINE_COLORS[line],
            showlegend=False
        ),
        row=1, col=idx
    )

fig.update_layout(
    title_text='호선별 개선 우선순위 Top 10',
    height=600
)

fig.show()

### 7.7 시간대별 배차간격 변화

In [47]:
# 시간대별 평균 배차간격 계산
hourly_data = []

for line in [2, 4, 5]:
    line_data = df_all[(df_all['호선'] == line) & (df_all['주중주말'] == 'DAY')].copy()
    
    for hour in range(5, 24):
        hour_data = line_data[line_data['시간대'] == hour]
        
        if len(hour_data) > 0:
            for (station, direction), group in hour_data.groupby(['역사명', '방향']):
                group = group.sort_values('열차도착시간_dt')
                if len(group) > 1:
                    headways = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
                    headways = headways[(headways > 1) & (headways < 60)]
                    if len(headways) > 0:
                        hourly_data.append({
                            '호선': line,
                            '시간대': hour,
                            '배차간격_분': headways.mean()
                        })

if hourly_data:
    df_hourly = pd.DataFrame(hourly_data)
    hourly_avg = df_hourly.groupby(['호선', '시간대'])['배차간격_분'].mean().reset_index()
    
    fig = px.line(
        hourly_avg,
        x='시간대',
        y='배차간격_분',
        color='호선',
        color_discrete_map=LINE_COLORS,
        title='시간대별 평균 배차간격 변화 (주중)',
        labels={'시간대': '시간대', '배차간격_분': '평균 배차간격 (분)'},
        markers=True
    )
    
    # 출퇴근 시간대 강조
    fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1,
                annotation_text="출근", annotation_position="top left")
    fig.add_vrect(x0=17, x1=19, fillcolor="blue", opacity=0.1,
                annotation_text="퇴근", annotation_position="top left")
    
    # X축 포맷 설정 (1시간 단위)
    fig.update_xaxes(
        tickmode='linear',
        tick0=5,
        dtick=1,
        range=[5, 24]
    )
    
    fig.update_layout(
        height=600,
        hovermode='x unified'
    )
    
    fig.show()
else:
    print('시간대별 데이터가 충분하지 않습니다.')

### 7.8 시간대별 혼잡도 변화

In [48]:
# 혼잡도 데이터 로드
CONGESTION_PATH = r'..\data\input\서울교통공사_지하철혼잡도정보_20250331.xlsx'
df_cong = pd.read_excel(CONGESTION_PATH)

# 2, 4, 5호선, 평일 데이터만 필터링
df_cong_filtered = df_cong[(df_cong['호선'].isin([2, 4, 5])) & (df_cong['요일구분'] == '평일')].copy()

print(f'✅ 혼잡도 데이터 로딩 완료')
print(f'   총 데이터: {len(df_cong_filtered):,}행')
print(f'   호선별 데이터 수:')
for line in [2, 4, 5]:
    count = len(df_cong_filtered[df_cong_filtered['호선'] == line])
    print(f'      {line}호선: {count:,}행')

# 시간대 컬럼 추출 (5시30분부터 23시30분까지)
time_cols = [col for col in df_cong_filtered.columns if '시' in col and '분' in col]
print(f'\n   시간대 컬럼 수: {len(time_cols)}개')

df_cong_filtered.head()

✅ 혼잡도 데이터 로딩 완료
   총 데이터: 271행
   호선별 데이터 수:
      2호선: 106행
      4호선: 52행
      5호선: 113행

   시간대 컬럼 수: 39개


,연번,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
20,21,평일,2,210,뚝섬,외선,8.5,16.6,12.7,37.8,...,53.6,39.3,36.2,42.9,44.0,29.3,20.1,15.0,6.3,3.5
21,22,평일,2,209,한양대,외선,10.7,12.8,13.3,37.9,...,57.6,44.5,38.4,47.8,48.0,37.1,20.5,15.9,8.9,3.5
22,23,평일,2,208,왕십리,외선,14.3,22.3,21.0,57.2,...,62.1,51.4,43.0,52.2,54.3,44.7,25.1,20.3,12.1,4.3
23,24,평일,2,207,상왕십리,외선,16.2,23.6,22.4,61.4,...,58.9,53.6,42.5,48.4,54.6,43.4,24.7,20.2,11.5,3.8
24,25,평일,2,206,신당,외선,18.4,26.2,21.5,62.2,...,56.3,51.8,41.9,45.7,54.6,45.1,24.9,18.9,10.5,6.0


In [49]:
# 시간대별 혼잡도 데이터 변환
congestion_hourly = []

for _, row in df_cong_filtered.iterrows():
    line = row['호선']
    for col in time_cols:
        # 시간 파싱 (예: '5시30분' -> 5.5)
        time_str = col.replace('시', ':').replace('분', '')
        hour_parts = time_str.split(':')
        hour = int(hour_parts[0])
        minute = int(hour_parts[1]) if len(hour_parts) > 1 else 0
        hour_decimal = hour + minute / 60.0
        
        congestion_value = row[col]
        if pd.notna(congestion_value):
            congestion_hourly.append({
                '호선': line,
                '시간': hour_decimal,
                '시간대_문자': col,
                '혼잡도': congestion_value
            })

df_cong_hourly = pd.DataFrame(congestion_hourly)

# 호선별, 시간대별 평균 혼잡도 계산
cong_avg = df_cong_hourly.groupby(['호선', '시간'])['혼잡도'].mean().reset_index()

print(f'✅ 시간대별 혼잡도 데이터 변환 완료: {len(df_cong_hourly):,}개 데이터 포인트')
print(f'   호선별 평균 혼잡도:')
for line in [2, 4, 5]:
    avg_cong = df_cong_hourly[df_cong_hourly['호선'] == line]['혼잡도'].mean()
    print(f'      {line}호선: {avg_cong:.1f}%')

cong_avg.head(10)

✅ 시간대별 혼잡도 데이터 변환 완료: 10,569개 데이터 포인트
   호선별 평균 혼잡도:
      2호선: 37.4%
      4호선: 32.5%
      5호선: 30.2%


,호선,시간,혼잡도
0,2,0.0,10.691509
1,2,0.5,6.786792
2,2,5.5,23.216038
3,2,6.0,23.666038
4,2,6.5,25.174528
5,2,7.0,38.922642
6,2,7.5,53.370755
7,2,8.0,68.464151
8,2,8.5,59.386792
9,2,9.0,46.179245


In [50]:
# 시간대별 혼잡도 변화 그래프
fig = px.line(
    cong_avg,
    x='시간',
    y='혼잡도',
    color='호선',
    color_discrete_map=LINE_COLORS,
    title='시간대별 평균 혼잡도 변화 (주중)',
    labels={'시간': '시간대', '혼잡도': '평균 혼잡도 (%)'},
    markers=True
)

# 출퇴근 시간대 강조
fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1,
            annotation_text="출근", annotation_position="top left")
fig.add_vrect(x0=17, x1=19, fillcolor="blue", opacity=0.1,
            annotation_text="퇴근", annotation_position="top left")

# X축 포맷 설정
fig.update_xaxes(
    tickmode='linear',
    tick0=5,
    dtick=1,
    range=[5, 24]
)

fig.update_layout(
    height=600,
    hovermode='x unified'
)

fig.show()

print('\n📊 시간대별 혼잡도 패턴 분석:')
print(f'   출근 시간대(7-9시) 평균 혼잡도:')
for line in [2, 4, 5]:
    morning_cong = cong_avg[(cong_avg['호선'] == line) & (cong_avg['시간'] >= 7) & (cong_avg['시간'] < 9)]['혼잡도'].mean()
    print(f'      {line}호선: {morning_cong:.1f}%')

print(f'\n   퇴근 시간대(17-19시) 평균 혼잡도:')
for line in [2, 4, 5]:
    evening_cong = cong_avg[(cong_avg['호선'] == line) & (cong_avg['시간'] >= 17) & (cong_avg['시간'] < 19)]['혼잡도'].mean()
    print(f'      {line}호선: {evening_cong:.1f}%')


📊 시간대별 혼잡도 패턴 분석:
   출근 시간대(7-9시) 평균 혼잡도:
      2호선: 55.0%
      4호선: 46.2%
      5호선: 48.0%

   퇴근 시간대(17-19시) 평균 혼잡도:
      2호선: 61.5%
      4호선: 52.9%
      5호선: 49.0%


## 8. 결과 저장

분석 결과를 CSV 파일로 저장합니다.

In [51]:
# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. 역간 소요시간
travel_stats.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_역간_소요시간.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_역간_소요시간.csv ({len(travel_stats):,}행)')

# 2. 배차간격
df_headway.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_배차간격.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_배차간격.csv ({len(df_headway):,}행)')

# 3. 복합지표 Top 50
df_combined.nlargest(50, '복합지표').to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_복합지표_Top50.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_복합지표_Top50.csv (50행)')

# 4. 개선 우선순위 Top 30
priority_top30.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_개선_우선순위_Top30.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_개선_우선순위_Top30.csv (30행)')

# 5. 종합 요약
df_line_summary.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_종합요약.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_종합요약.csv (3행)')

print(f'\n📁 모든 파일이 저장되었습니다: {OUTPUT_DIR}')

✅ 저장: 2_4_5호선_역간_소요시간.csv (1,071행)
✅ 저장: 2_4_5호선_배차간격.csv (933행)
✅ 저장: 2_4_5호선_복합지표_Top50.csv (50행)
✅ 저장: 2_4_5호선_개선_우선순위_Top30.csv (30행)
✅ 저장: 2_4_5호선_종합요약.csv (3행)

📁 모든 파일이 저장되었습니다: ..\data\output\4_2_4_5호선_종합


## 9. 분석 결론

### 주요 발견사항

1. **복합지표 분석**
   - 복합지표 = 소요시간 × 배차간격
   - 높은 복합지표 = 긴 이동시간 + 긴 대기시간 = 승객 불편 증가
   - 서비스 개선이 시급한 구간 식별

2. **호선별 특성**
   - 각 호선의 평균 소요시간, 배차간격, 복합지표 비교
   - 호선별 개선 필요 구간 우선순위 도출

3. **개선 방안**
   - **배차 증편 우선**: 배차간격이 긴 구간
   - **선로/신호 개선 우선**: 소요시간이 긴 구간
   - **종합 개선 필요**: 두 지표가 모두 높은 구간

4. **시간대별 패턴**
   - 출퇴근 시간대 배차간격 단축
   - 심야 시간대 배차간격 확대
   - 시간대별 탄력적 운영 필요

### 활용 방안
- 배차 계획 수립 시 우선순위 참고
- 역 시설 개선 우선순위 결정
- 혼잡도 예측 모델의 입력 변수
- 승객 만족도 개선 전략 수립

---

**분석 완료**: 2/4/5호선 종합 분석이 성공적으로 완료되었습니다.

## 10. 호선별 혼잡도가 가장 높은 역 분석

In [52]:
# 각 역별 평균 혼잡도 계산
station_congestion = []

for _, row in df_cong_filtered.iterrows():
    line = row['호선']
    station = row['출발역']
    direction = row['상하구분']
    
    # 시간대별 혼잡도 평균 계산
    congestion_values = [row[col] for col in time_cols if pd.notna(row[col])]
    if congestion_values:
        avg_congestion = np.mean(congestion_values)
        max_congestion = np.max(congestion_values)
        
        station_congestion.append({
            '호선': line,
            '역명': station,
            '방향': direction,
            '평균혼잡도': avg_congestion,
            '최대혼잡도': max_congestion
        })

df_station_cong = pd.DataFrame(station_congestion)

# 역별로 그룹핑 (방향 통합하여 평균)
station_avg = df_station_cong.groupby(['호선', '역명']).agg({
    '평균혼잡도': 'mean',
    '최대혼잡도': 'max'
}).round(2).reset_index()

print('✅ 역별 혼잡도 계산 완료')
print(f'   총 역 수: {len(station_avg):,}개')
print(f'   호선별 역 수:')
for line in [2, 4, 5]:
    count = len(station_avg[station_avg['호선'] == line])
    print(f'      {line}호선: {count}개')

print(f'\n📊 호선별 혼잡도가 가장 높은 역 (평균 혼잡도 기준):')
print('=' * 80)

for line in [2, 4, 5]:
    line_data = station_avg[station_avg['호선'] == line].nlargest(1, '평균혼잡도')
    if not line_data.empty:
        station_name = line_data.iloc[0]['역명']
        avg_cong = line_data.iloc[0]['평균혼잡도']
        max_cong = line_data.iloc[0]['최대혼잡도']
        print(f'{line}호선: {station_name} (평균 {avg_cong:.1f}%, 최대 {max_cong:.1f}%)')

print('\n' + '=' * 80)
print(f'\n📊 호선별 혼잡도 Top 5:')
for line in [2, 4, 5]:
    print(f'\n{line}호선:')
    line_top5 = station_avg[station_avg['호선'] == line].nlargest(5, '평균혼잡도')
    for idx, row in line_top5.iterrows():
        print(f'   {row["역명"]}: 평균 {row["평균혼잡도"]:.1f}%, 최대 {row["최대혼잡도"]:.1f}%')

station_avg.head(20)

✅ 역별 혼잡도 계산 완료
   총 역 수: 133개
   호선별 역 수:
      2호선: 51개
      4호선: 26개
      5호선: 56개

📊 호선별 혼잡도가 가장 높은 역 (평균 혼잡도 기준):
2호선: 방배 (평균 56.0%, 최대 142.3%)
4호선: 동대문 (평균 51.5%, 최대 148.5%)
5호선: 길동 (평균 51.0%, 최대 128.0%)


📊 호선별 혼잡도 Top 5:

2호선:
   방배: 평균 56.0%, 최대 142.3%
   서초: 평균 55.8%, 최대 140.0%
   교대: 평균 55.6%, 최대 132.3%
   사당: 평균 54.2%, 최대 147.4%
   강남: 평균 53.7%, 최대 132.2%

4호선:
   동대문: 평균 51.5%, 최대 148.5%
   혜화: 평균 49.2%, 최대 140.7%
   남태령: 평균 48.9%, 최대 133.2%
   동대문역사문화공원: 평균 46.9%, 최대 128.2%
   한성대입구: 평균 46.0%, 최대 143.5%

5호선:
   길동: 평균 51.0%, 최대 128.0%
   아차산: 평균 47.9%, 최대 111.7%
   굽은다리: 평균 46.8%, 최대 113.7%
   광나루: 평균 46.2%, 최대 107.7%
   군자: 평균 44.3%, 최대 116.1%


,호선,역명,평균혼잡도,최대혼잡도
0,2,강남,53.68,132.2
1,2,강변,30.87,66.2
2,2,건대입구,32.26,74.7
3,2,교대,55.57,132.3
4,2,구로디지털단지,42.14,91.3
5,2,구의,31.30,62.5
6,2,까치산,8.70,59.7
7,2,낙성대,51.19,129.1
8,2,당산,45.16,115.0
9,2,대림,44.24,90.2


### 10.1 호선별 혼잡도 Top 10 시각화

In [53]:
# 호선별 Top 10 혼잡도 역
fig = go.Figure()

for line in [2, 4, 5]:
    line_data = station_avg[station_avg['호선'] == line].nlargest(10, '평균혼잡도')
    fig.add_trace(go.Bar(
        name=f'{line}호선',
        y=line_data['역명'],
        x=line_data['평균혼잡도'],
        orientation='h',
        marker_color=LINE_COLORS[line],
        text=line_data['평균혼잡도'].round(1),
        textposition='outside'
    ))

fig.update_layout(
    title='호선별 평균 혼잡도 Top 10 (주중)',
    xaxis_title='평균 혼잡도 (%)',
    yaxis_title='역명',
    height=800,
    barmode='group'
)

fig.show()

### 10.2 호선별 혼잡도 분포

In [54]:
# 호선별 혼잡도 박스플롯
fig = px.box(
    station_avg,
    x='호선',
    y='평균혼잡도',
    color='호선',
    color_discrete_map=LINE_COLORS,
    title='호선별 혼잡도 분포 (주중)',
    labels={'평균혼잡도': '평균 혼잡도 (%)', '호선': '호선'},
    points='all'
)

fig.update_layout(
    height=600,
    showlegend=False
)

fig.show()

# 호선별 통계
print('\n📊 호선별 혼잡도 통계:')
for line in [2, 4, 5]:
    line_data = station_avg[station_avg['호선'] == line]['평균혼잡도']
    print(f'\n{line}호선:')
    print(f'   평균: {line_data.mean():.1f}%')
    print(f'   중앙값: {line_data.median():.1f}%')
    print(f'   최소: {line_data.min():.1f}%')
    print(f'   최대: {line_data.max():.1f}%')
    print(f'   표준편차: {line_data.std():.1f}%')


📊 호선별 혼잡도 통계:

2호선:
   평균: 37.9%
   중앙값: 40.8%
   최소: 6.7%
   최대: 56.0%
   표준편차: 11.9%

4호선:
   평균: 32.5%
   중앙값: 30.9%
   최소: 5.6%
   최대: 51.5%
   표준편차: 11.8%

5호선:
   평균: 30.1%
   중앙값: 33.8%
   최소: 1.4%
   최대: 51.0%
   표준편차: 12.7%


### 10.3 혼잡도 분석 결과 저장

In [55]:
# 역별 혼잡도 전체 저장
station_avg.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_역별_혼잡도.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_역별_혼잡도.csv ({len(station_avg):,}행)')

# 호선별 Top 10 혼잡도 역 저장
top10_stations = []
for line in [2, 4, 5]:
    line_top10 = station_avg[station_avg['호선'] == line].nlargest(10, '평균혼잡도')
    top10_stations.append(line_top10)

df_top10_stations = pd.concat(top10_stations, ignore_index=True)
df_top10_stations.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_혼잡도_Top10.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_혼잡도_Top10.csv ({len(df_top10_stations):,}행)')

# 호선별 가장 혼잡한 역 요약
highest_cong = []
for line in [2, 4, 5]:
    line_data = station_avg[station_avg['호선'] == line].nlargest(1, '평균혼잡도')
    if not line_data.empty:
        highest_cong.append(line_data)

df_highest = pd.concat(highest_cong, ignore_index=True)
df_highest.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_최고_혼잡_역.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_최고_혼잡_역.csv ({len(df_highest):,}행)')

print(f'\n📁 혼잡도 분석 결과가 저장되었습니다: {OUTPUT_DIR}')

✅ 저장: 2_4_5호선_역별_혼잡도.csv (133행)
✅ 저장: 2_4_5호선_혼잡도_Top10.csv (30행)
✅ 저장: 2_4_5호선_최고_혼잡_역.csv (3행)

📁 혼잡도 분석 결과가 저장되었습니다: ..\data\output\4_2_4_5호선_종합


### 10.4 시간대별 최고 혼잡도 역 분석

In [56]:
# 시간대별 최고 혼잡도 역 찾기
time_station_max = []

for time_col in time_cols:
    # 시간 파싱
    time_str = time_col.replace('시', ':').replace('분', '')
    hour_parts = time_str.split(':')
    hour = int(hour_parts[0])
    minute = int(hour_parts[1]) if len(hour_parts) > 1 else 0
    hour_decimal = hour + minute / 60.0
    
    for line in [2, 4, 5]:
        line_data = df_cong_filtered[df_cong_filtered['호선'] == line]
        
        if len(line_data) > 0 and time_col in line_data.columns:
            # 해당 시간대에 가장 혼잡한 역 찾기
            max_idx = line_data[time_col].idxmax()
            if pd.notna(max_idx):
                max_row = line_data.loc[max_idx]
                max_congestion = max_row[time_col]
                
                if pd.notna(max_congestion):
                    time_station_max.append({
                        '호선': line,
                        '시간': hour_decimal,
                        '시간대_문자': time_col,
                        '역명': max_row['출발역'],
                        '방향': max_row['상하구분'],
                        '혼잡도': max_congestion
                    })

df_time_max = pd.DataFrame(time_station_max)

print(f'✅ 시간대별 최고 혼잡도 역 계산 완료: {len(df_time_max):,}개 데이터 포인트')
print(f'\n📊 주요 시간대별 최고 혼잡도 역:')
print('=' * 100)

# 주요 시간대별로 출력
key_times = [7.5, 8.0, 8.5, 17.5, 18.0, 18.5]  # 출퇴근 시간
for time in key_times:
    print(f'\n{int(time)}시 {int((time % 1) * 60)}분:')
    time_data = df_time_max[df_time_max['시간'] == time]
    for line in [2, 4, 5]:
        line_data = time_data[time_data['호선'] == line]
        if not line_data.empty:
            row = line_data.iloc[0]
            print(f'  {line}호선: {row["역명"]} ({row["방향"]}) - {row["혼잡도"]:.1f}%')

df_time_max.head(20)

✅ 시간대별 최고 혼잡도 역 계산 완료: 117개 데이터 포인트

📊 주요 시간대별 최고 혼잡도 역:

7시 30분:
  2호선: 사당 (외선) - 124.2%
  4호선: 한성대입구 (하선) - 119.8%
  5호선: 양평 (하선) - 122.1%

8시 0분:
  2호선: 사당 (외선) - 131.1%
  4호선: 성신여대입구 (하선) - 134.0%
  5호선: 신길 (하선) - 129.8%

8시 30분:
  2호선: 사당 (외선) - 146.8%
  4호선: 동대문 (하선) - 148.5%
  5호선: 길동 (상선) - 107.4%

17시 30분:
  2호선: 방배 (내선) - 140.1%
  4호선: 동대문 (상선) - 121.5%
  5호선: 길동 (하선) - 128.0%

18시 0분:
  2호선: 방배 (내선) - 142.3%
  4호선: 동대문 (상선) - 118.6%
  5호선: 길동 (하선) - 125.9%

18시 30분:
  2호선: 방배 (내선) - 137.4%
  4호선: 동대문 (상선) - 135.2%
  5호선: 길동 (하선) - 115.1%


,호선,시간,시간대_문자,역명,방향,혼잡도
0,2,5.5,5시30분,대림,내선,87.4
1,4,5.5,5시30분,성신여대입구,하선,69.7
2,5,5.5,5시30분,양평,하선,44.3
3,2,6.0,6시00분,사당,외선,60.5
4,4,6.0,6시00분,한성대입구,하선,47.4
5,5,6.0,6시00분,길동,상선,40.8
6,2,6.5,6시30분,낙성대,외선,65.0
7,4,6.5,6시30분,성신여대입구,하선,56.8
8,5,6.5,6시30분,길동,상선,71.8
9,2,7.0,7시00분,사당,외선,106.3


### 10.5 시간대별 최고 혼잡도 역 시각화

In [57]:
# 1. 시간대별 최고 혼잡도 추이 (라인 차트 + 역명 표시)
fig = go.Figure()

# 각 호선별 최고 혼잡도 지점 저장
max_points = []

for line in [2, 4, 5]:
    line_data = df_time_max[df_time_max['호선'] == line].copy()
    line_data = line_data.sort_values('시간')
    
    # 라인 차트
    fig.add_trace(go.Scatter(
        x=line_data['시간'],
        y=line_data['혼잡도'],
        mode='lines+markers',
        name=f'{line}호선',
        line=dict(color=LINE_COLORS[line], width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{text}</b><br>혼잡도: %{y:.1f}%<extra></extra>',
        text=line_data['역명']
    ))
    
    # 최고 혼잡도 지점 찾기
    max_idx = line_data['혼잡도'].idxmax()
    max_row = line_data.loc[max_idx]
    max_points.append({
        'line': line,
        'time': max_row['시간'],
        'congestion': max_row['혼잡도'],
        'station': max_row['역명'],
        'color': LINE_COLORS[line]
    })

# 출퇴근 시간대 강조
fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1,
            annotation_text="출근 시간대", annotation_position="top left")
fig.add_vrect(x0=17, x1=19, fillcolor="blue", opacity=0.1,
            annotation_text="퇴근 시간대", annotation_position="top left")

# 각 호선별 최고 혼잡도 지점에 annotation 추가
for point in max_points:
    # 2호선은 우측, 나머지는 위쪽에 배치
    if point['line'] == 2:
        ax, ay = 50, 0  # 우측
    elif point['line'] == 5:
        ax, ay = -50, 0  # 좌측
    else:
        ax, ay = 0, -40  # 위쪽
    
    fig.add_annotation(
        x=point['time'],
        y=point['congestion'],
        text=f"{point['station']}<br>{point['congestion']:.1f}%",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor=point['color'],
        ax=ax,
        ay=ay,
        bgcolor='white',
        bordercolor=point['color'],
        borderwidth=2,
        borderpad=4,
        font=dict(size=10, color=point['color'])
    )

fig.update_layout(
    title='시간대별 최고 혼잡도 역 추이 (주중)<br><sub>각 시간대별로 가장 혼잡한 역의 혼잡도</sub>',
    xaxis_title='시간대',
    yaxis_title='최고 혼잡도 (%)',
    height=600,
    hovermode='x unified',
    xaxis=dict(
        tickmode='linear',
        tick0=5,
        dtick=1,
        range=[5, 24]
    )
)

fig.show()

In [58]:
# 2. 호선별 히트맵 (시간대 × 최고 혼잡도 역)
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=['2호선', '4호선', '5호선'],
    vertical_spacing=0.08,
    row_heights=[0.33, 0.33, 0.33]
)

for idx, line in enumerate([2, 4, 5], 1):
    line_data = df_time_max[df_time_max['호선'] == line].copy()
    
    # 피벗 테이블 생성 (시간대별 역명과 혼잡도)
    # 각 시간대마다 하나의 역만 있으므로 단순화
    line_data = line_data.sort_values('시간')
    
    # 히트맵 데이터 준비
    z_values = [line_data['혼잡도'].tolist()]
    x_labels = [f"{int(t)}:{int((t%1)*60):02d}" for t in line_data['시간']]
    hover_text = [[f"{row['역명']} ({row['방향']})<br>혼잡도: {row['혼잡도']:.1f}%" 
                   for _, row in line_data.iterrows()]]
    
    fig.add_trace(
        go.Heatmap(
            z=z_values,
            x=x_labels,
            y=['최고 혼잡역'],
            colorscale='Reds',
            text=[[row['역명'] for _, row in line_data.iterrows()]],
            texttemplate='%{text}',
            textfont=dict(size=8),
            hovertext=hover_text,
            hoverinfo='text',
            colorbar=dict(
                title='혼잡도 (%)',
                len=0.3,
                y=1 - (idx-1)*0.35
            ),
            showscale=(idx == 1)  # 첫 번째만 컬러바 표시
        ),
        row=idx, col=1
    )
    
    # X축 레이블 간격 조정
    fig.update_xaxes(
        tickangle=-45,
        tickmode='array',
        tickvals=list(range(0, len(x_labels), 3)),
        ticktext=[x_labels[i] for i in range(0, len(x_labels), 3)],
        row=idx, col=1
    )

fig.update_layout(
    title_text='시간대별 최고 혼잡도 역 히트맵 (주중)',
    height=800,
    showlegend=False
)

fig.show()

In [59]:
# 3. 피크 시간대 최고 혼잡도 역 비교 (막대 그래프)
peak_times = {
    '출근 피크 (8:00)': 8.0,
    '퇴근 피크 (18:00)': 18.0
}

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=list(peak_times.keys()),
    horizontal_spacing=0.15
)

for col_idx, (label, time) in enumerate(peak_times.items(), 1):
    time_data = df_time_max[df_time_max['시간'] == time]
    
    # 호선별 데이터
    lines = []
    stations = []
    congestions = []
    colors = []
    
    for line in [2, 4, 5]:
        line_data = time_data[time_data['호선'] == line]
        if not line_data.empty:
            row = line_data.iloc[0]
            lines.append(f"{line}호선")
            stations.append(f"{row['역명']}")
            congestions.append(row['혼잡도'])
            colors.append(LINE_COLORS[line])
    
    fig.add_trace(
        go.Bar(
            x=lines,
            y=congestions,
            marker_color=colors,
            text=[f"{s}<br>{c:.1f}%" for s, c in zip(stations, congestions)],
            textposition='outside',
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>역: %{text}<extra></extra>'
        ),
        row=1, col=col_idx
    )
    
    fig.update_yaxes(title_text='혼잡도 (%)', row=1, col=col_idx)
    fig.update_xaxes(title_text='호선', row=1, col=col_idx)

fig.update_layout(
    title_text='피크 시간대별 최고 혼잡도 역 비교',
    height=500,
    showlegend=False
)

fig.show()

In [60]:
# 4. 가장 자주 최고 혼잡도를 기록하는 역 분석
station_frequency = df_time_max.groupby(['호선', '역명']).size().reset_index(name='빈도')
station_frequency = station_frequency.sort_values(['호선', '빈도'], ascending=[True, False])

print('📊 시간대별 최고 혼잡도를 가장 자주 기록한 역 Top 5:')
print('=' * 80)

for line in [2, 4, 5]:
    print(f'\n{line}호선:')
    line_freq = station_frequency[station_frequency['호선'] == line].head(5)
    total_times = len(df_time_max[df_time_max['호선'] == line]['시간'].unique())
    for idx, row in line_freq.iterrows():
        percentage = (row['빈도'] / total_times * 100) if total_times > 0 else 0
        print(f'  {row["역명"]}: {row["빈도"]}회 ({percentage:.1f}%)')

# 시각화
fig = go.Figure()

for line in [2, 4, 5]:
    line_freq = station_frequency[station_frequency['호선'] == line].head(10)
    fig.add_trace(go.Bar(
        name=f'{line}호선',
        y=line_freq['역명'],
        x=line_freq['빈도'],
        orientation='h',
        marker_color=LINE_COLORS[line],
        text=line_freq['빈도'],
        textposition='outside'
    ))

fig.update_layout(
    title='시간대별 최고 혼잡도 기록 빈도 Top 10 (호선별)',
    xaxis_title='기록 횟수',
    yaxis_title='역명',
    height=800,
    barmode='group'
)

fig.show()

📊 시간대별 최고 혼잡도를 가장 자주 기록한 역 Top 5:

2호선:
  방배: 13회 (33.3%)
  사당: 12회 (30.8%)
  서초: 5회 (12.8%)
  대림: 2회 (5.1%)
  강남: 1회 (2.6%)

4호선:
  동대문: 14회 (35.9%)
  남태령: 7회 (17.9%)
  혜화: 6회 (15.4%)
  성신여대입구: 5회 (12.8%)
  동대문역사문화공원: 4회 (10.3%)

5호선:
  길동: 32회 (82.1%)
  양평: 3회 (7.7%)
  영등포구청: 3회 (7.7%)
  신길: 1회 (2.6%)


In [61]:
# 시간대별 최고 혼잡도 역 결과 저장
df_time_max.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_시간대별_최고혼잡역.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_시간대별_최고혼잡역.csv ({len(df_time_max):,}행)')

station_frequency.to_csv(
    os.path.join(OUTPUT_DIR, '2_4_5호선_최고혼잡역_빈도.csv'),
    index=False,
    encoding='utf-8-sig'
)
print(f'✅ 저장: 2_4_5호선_최고혼잡역_빈도.csv ({len(station_frequency):,}행)')

print(f'\n📁 시간대별 분석 결과가 저장되었습니다: {OUTPUT_DIR}')

✅ 저장: 2_4_5호선_시간대별_최고혼잡역.csv (117행)
✅ 저장: 2_4_5호선_최고혼잡역_빈도.csv (21행)

📁 시간대별 분석 결과가 저장되었습니다: ..\data\output\4_2_4_5호선_종합
